In [ ]:
import pandas as pd
import numpy as np
import torch
from google.colab import files
uploaded = files.upload()

Saving FinancialSentiment.csv to FinancialSentiment.csv


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
data = pd.read_csv("FinancialSentiment.csv", header = None, encoding="ISO-8859-1")
data.columns = ['Label', 'Text']
data.head()


,Label,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [ ]:
data.Label.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [ ]:
data.shape

(4846, 2)

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)
print(data_test.shape)

(485, 2)


**Finbert**

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 14.0MB/s 
     |████████████████████████████████| 3.0MB 21.6MB/s 
     |████████████████████████████████| 890kB 34.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=2df541512510e411211758d0daca30751eb2a615ea89194d7b2a7f617a372a50
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")

model = AutoModelForSequenceClassification.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")

In [ ]:
MAX_LEN = 160
class_names = ['negative', 'neutral', 'positive']
results = []
for i in data.Text:
  encoded_new = tokenizer.encode_plus(
                        i,                      # Sentence to encode.
                        add_special_tokens = True,        # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,             # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,     # Construct attn. masks.
                        return_tensors = 'pt',            # Return pytorch tensors.
                   )

  # Add the encoded sentence to the list.    
  input_idst = (encoded_new['input_ids'])
  attention_maskst = (encoded_new['attention_mask'])

  # Convert the lists into tensors.
  input_idst = torch.cat([input_idst], dim=0)
  attention_maskst = torch.cat([attention_maskst], dim=0)


  new_test_output = model(input_idst, token_type_ids=None, 
                      attention_mask=attention_maskst)

  logits = new_test_output[0]
  predicted = logits.detach().numpy()

  # Store predictions
  flat_predictions = np.concatenate(predicted, axis=0)

  # For each sample, pick the label (0 or 1) with the higher score.
  new_predictions = np.argmax(flat_predictions).flatten()

  results.append(class_names[new_predictions[0]])
print(results)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

['neutral', 'neutral', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'positive'

In [ ]:
from sklearn.metrics import accuracy_score
trueVal = []
predictVal = []
for i in data.Label:
  if i == "neutral":
    trueVal.append(0)
  elif i == "negative":
    trueVal.append(-1)
  elif i == "positive":
    trueVal.append(1)

for i in results:
  if i == "neutral":
    predictVal.append(0)
  elif i == "negative":
    predictVal.append(-1)
  elif i == "positive":
    predictVal.append(1)

In [ ]:
print("Accuracy of FinBERT: " + str(sum(data.Label==results)/len(results)))
print("Total number of data points: " + str(data.Label.shape[0]))
print("Number of incorrectly classified data points: " + str(sum(data.Label != results)))

Accuracy of FinBERT: 0.9989682212133718
Total number of data points: 4846
Number of incorrectly classified data points: 5


Note: The code used to evaluate the sentiment analysis of all the data points with the pre-trained FinBERT is extremely inefficient since it does a for loop through all the points and individually calculating the sentiment. I don't know if there is a better way since there is not sufficient documentation on FinBERT. Regardless of efficiency, however, the FinBERT classification is very effective.